In [ ]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import PowerTransformer

# Carregar o dataset Iris
iris = datasets.load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# Adicionar a coluna da espécie
df["species"] = iris.target
df["species"] = df["species"].map({0: "setosa", 1: "versicolor", 2: "virginica"})

# Separar os grupos por espécie
species_groups = df.groupby("species")

# Verificar normalidade e aplicar normalização se necessário
normalizado = False
for column in iris.feature_names:
    p_values = []
    for specie in df["species"].unique():
        grupo = species_groups[column].get_group(specie)
        stat, p = stats.shapiro(grupo)
        p_values.append(p)
    
    # Se algum p-valor for menor que 0.05, rejeitamos a normalidade
    if any(p < 0.05 for p in p_values):
        normalizado = True
        print(f"🔸 {column} não segue distribuição normal. Aplicando transformação Yeo-Johnson...")
        pt = PowerTransformer(method="yeo-johnson")
        df[column] = pt.fit_transform(df[[column]])

# Aplicar ANOVA para todas as características
anova_results = {}

for column in iris.feature_names:
    grupos = [species_groups[column].get_group(specie) for specie in df["species"].unique()]
    stat, p_value = stats.f_oneway(*grupos)
    anova_results[column] = {"F-Estatística": stat, "p-Valor": p_value}

# Exibir os resultados
for feature, values in anova_results.items():
    print(f"\n🔹 {feature}:")
    print(f"   - Estatística F: {values['F-Estatística']:.4f}")
    print(f"   - p-Valor: {values['p-Valor']:.10f}")

if normalizado:
    print("\n⚠️ Algumas variáveis foram transformadas para garantir normalidade.")



🔹 sepal length (cm):
   - Estatística F: 119.2645
   - p-Valor: 0.0000000000

🔹 sepal width (cm):
   - Estatística F: 49.1600
   - p-Valor: 0.0000000000

🔹 petal length (cm):
   - Estatística F: 1180.1612
   - p-Valor: 0.0000000000

🔹 petal width (cm):
   - Estatística F: 960.0071
   - p-Valor: 0.0000000000


In [ ]:
import pandas as pd

# URLs dos datasets
url_red = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
url_white = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

# Carregar os datasets
df_red = pd.read_csv(url_red, sep=';')
df_white = pd.read_csv(url_white, sep=';')

# Adicionar coluna indicando o tipo de vinho
df_red['type'] = 'red'
df_white['type'] = 'white'

# Combinar os datasets
df = pd.concat([df_red, df_white], ignore_index=True)
# Exibir as primeiras linhas do DataFrame
print(df.head())

# Verificar valores ausentes
display(df.isnull().sum())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality type  
0      9.4        5  red  
1      9.8        5  red  
2    

,0
fixed acidity,0
volatile acidity,0
citric acid,0
residual sugar,0
chlorides,0
free sulfur dioxide,0
total sulfur dioxide,0
density,0
pH,0
sulphates,0


In [ ]:
import scipy.stats as stats

# Separar os dados por tipo de vinho
red_wine = df[df['type'] == 'red']
white_wine = df[df['type'] == 'white']

# Características a serem analisadas
features = df.columns[:-2]  # Excluindo 'quality' e 'type'

# Aplicar ANOVA para cada característica
anova_results = {}
for feature in features:
    stat, p_value = stats.f_oneway(red_wine[feature], white_wine[feature])
    anova_results[feature] = {'F-Estatística': stat, 'p-Valor': p_value}

# Exibir os resultados
for feature, result in anova_results.items():
    print(f"\n🔹 {feature}:")
    print(f"   - Estatística F: {result['F-Estatística']:.4f}")
    print(f"   - p-Valor: {result['p-Valor']:.10f}")



🔹 fixed acidity:
   - Estatística F: 2016.5101
   - p-Valor: 0.0000000000

🔹 volatile acidity:
   - Estatística F: 4829.3171
   - p-Valor: 0.0000000000

🔹 citric acid:
   - Estatística F: 236.3892
   - p-Valor: 0.0000000000

🔹 residual sugar:
   - Estatística F: 899.7663
   - p-Valor: 0.0000000000

🔹 chlorides:
   - Estatística F: 2315.8295
   - p-Valor: 0.0000000000

🔹 free sulfur dioxide:
   - Estatística F: 1858.1364
   - p-Valor: 0.0000000000

🔹 total sulfur dioxide:
   - Estatística F: 6252.7956
   - p-Valor: 0.0000000000

🔹 density:
   - Estatística F: 1169.6552
   - p-Valor: 0.0000000000

🔹 pH:
   - Estatística F: 789.0495
   - p-Valor: 0.0000000000

🔹 sulphates:
   - Estatística F: 2021.7075
   - p-Valor: 0.0000000000

🔹 alcohol:
   - Estatística F: 7.0677
   - p-Valor: 0.0078678740


In [ ]:
# Teste para verificar a qualidade de safras de um tipo de vinho
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# URL do dataset de vinho tinto
url_red = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'

# Carregar o dataset
red_wine = pd.read_csv(url_red, sep=';')

# Simular diferentes safras com base em faixas de densidade (como proxy para safra)
red_wine['safra'] = pd.qcut(red_wine['density'], q=3, labels=['Safra A', 'Safra B', 'Safra C'])

# Definir um número significativo de amostras por safra (exemplo: 150 por safra)
n_samples = 150
np.random.seed(42)  # Para reprodutibilidade

safra_a_sample = red_wine[red_wine['safra'] == 'Safra A'].sample(n=n_samples, random_state=42)
safra_b_sample = red_wine[red_wine['safra'] == 'Safra B'].sample(n=n_samples, random_state=42)
safra_c_sample = red_wine[red_wine['safra'] == 'Safra C'].sample(n=n_samples, random_state=42)

# Criar um DataFrame com todas as amostras combinadas
df_tukey = pd.concat([
    safra_a_sample.assign(Safra='Safra A')[['quality', 'Safra']],
    safra_b_sample.assign(Safra='Safra B')[['quality', 'Safra']],
    safra_c_sample.assign(Safra='Safra C')[['quality', 'Safra']]
])

# Aplicar o teste ANOVA (one-way ANOVA)
stat, p_value = stats.f_oneway(
    safra_a_sample['quality'], safra_b_sample['quality'], safra_c_sample['quality']
)

# Exibir os resultados
print(f'Estatística F: {stat:.4f}')
print(f'Valor-p: {p_value:.4f}')

# Interpretar o resultado da ANOVA
alpha = 0.05
if p_value < alpha:
    print("Rejeitamos H0: Existe uma diferença significativa entre as qualidades do vinho entre as safras.")

    # Aplicar o teste de Tukey HSD
    tukey_result = pairwise_tukeyhsd(df_tukey['quality'], df_tukey['Safra'], alpha=0.05)
    print("\nResultados do Teste de Tukey:")
    print(tukey_result)
else:
    print("Falhamos em rejeitar H0: Não há evidências suficientes para afirmar que as safras influenciam na qualidade do vinho.")


Estatística F: 11.3597
Valor-p: 0.0000
Rejeitamos H0: Existe uma diferença significativa entre as qualidades do vinho entre as safras.

Resultados do Teste de Tukey:
 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
 group1  group2 meandiff p-adj   lower   upper  reject
------------------------------------------------------
Safra A Safra B  -0.2867 0.0051 -0.5013 -0.0721   True
Safra A Safra C  -0.4267    0.0 -0.6413 -0.2121   True
Safra B Safra C    -0.14 0.2759 -0.3546  0.0746  False
------------------------------------------------------
